### Análise de eleição - Eduardo Paes (55)

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(database='databases/eleicoes_2024_rj.sql')
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'databases/eleicoes_2024.sql' AS geral")
cursor.execute("ATTACH DATABASE 'databases/eleicoes_2024_rj.sql' AS rj")

In [3]:
cursor.execute("PRAGMA database_list;")
databases = cursor.fetchall()
print("Bancos de dados anexados:", databases)

Bancos de dados anexados: [(0, 'main', 'd:\\projects\\tse-data\\databases\\eleicoes_2024_rj.sql'), (2, 'geral', 'd:\\projects\\tse-data\\databases\\eleicoes_2024.sql'), (3, 'rj', 'd:\\projects\\tse-data\\databases\\eleicoes_2024_rj.sql')]


In [4]:
index_resultados = """
    CREATE INDEX IF NOT EXISTS idx_resultados 
    ON resultados(NM_VOTAVEL, NR_VOTAVEL, NR_LOCAL_VOTACAO,NR_ZONA,NR_SECAO,CD_MUNICIPIO, SG_UF);
"""
index_eleitorado = """
    CREATE INDEX IF NOT EXISTS idx_eleitorado 
    ON eleitorado(NR_LOCAL_VOTACAO,NR_ZONA,NR_SECAO,CD_MUNICIPIO, SG_UF);
"""
cursor.execute(index_resultados)
cursor.execute(index_eleitorado)
conn.commit()

r.DS_CARGO, 
r.NM_VOTAVEL, 
r.NR_SECAO, 
l.NM_MUNICIPIO, 
l.NM_BAIRRO,
l.NR_LATITUDE,
l.NR_LONGITUDE,  
e.DS_GENERO, 
e.DS_RACA_COR, 
e.DS_ESTADO_CIVIL, 
e.DS_FAIXA_ETARIA, 
e.DS_GRAU_ESCOLARIDADE, 
r.QT_VOTOS, 
l.QT_ELEITOR_SECAO, 
e.QT_ELEITORES_PERFIL

In [5]:
query = """
    SELECT 
        r.DS_CARGO, 
        r.NM_VOTAVEL, 
        r.NR_SECAO, 
        l.NM_MUNICIPIO, 
        l.NM_BAIRRO,
        l.NR_LATITUDE,
        l.NR_LONGITUDE, 
        e.QT_ELEITORES_PERFIL
    FROM 
        rj.resultados AS r
    INNER JOIN 
        geral.locais AS l
        ON 
            r.NR_LOCAL_VOTACAO = l.NR_LOCAL_VOTACAO
            AND r.NR_ZONA = l.NR_ZONA
            AND r.NR_SECAO = l.NR_SECAO
            AND r.SG_UF = l.SG_UF
            AND r.CD_MUNICIPIO = l.CD_MUNICIPIO
    INNER JOIN 
        rj.eleitorado AS e
        ON 
            r.NR_LOCAL_VOTACAO = e.NR_LOCAL_VOTACAO
            AND r.NR_ZONA = e.NR_ZONA
            AND r.NR_SECAO = e.NR_SECAO
            AND r.SG_UF = e.SG_UF
            AND r.CD_MUNICIPIO = e.CD_MUNICIPIO 
    WHERE 
        NM_VOTAVEL LIKE '%EDUARDO%PAES%';
"""

In [6]:
result = pd.read_sql_query(query, conn)

In [7]:
result.head(1)

,DS_CARGO,NM_VOTAVEL,NR_SECAO,NM_MUNICIPIO,NM_BAIRRO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL
0,Prefeito,EDUARDO DA COSTA PAES,254,RIO DE JANEIRO,CAMPO GRANDE,-22.890643,-43.563465,1


In [9]:
votos_bairro = result.groupby(by='NM_BAIRRO').agg(
    {
        'QT_ELEITORES_PERFIL': 'sum',
        'NR_LATITUDE': 'mean',
        'NR_LONGITUDE': 'mean',
    }
).sort_values(by='QT_ELEITORES_PERFIL', ascending=False)
votos_bairro

,QT_ELEITORES_PERFIL,NR_LATITUDE,NR_LONGITUDE
NM_BAIRRO,,,
CAMPO GRANDE,293901,-22.898534,-43.564016
SANTA CRUZ,162417,-22.920209,-43.684543
TIJUCA,151213,-22.927706,-43.233329
REALENGO,142862,-22.877801,-43.430318
BANGU,128301,-22.877016,-43.469724
...,...,...,...
FUNDÃO,1353,-22.860359,-43.218952
TUBIACANGA,1260,-22.785850,-43.225929
JACAREPAGUÁ,1245,-22.968071,-43.392573


In [10]:
import folium
from folium.plugins import HeatMap

In [11]:
mapa = folium.Map(location=[votos_bairro["NR_LATITUDE"].mean(), votos_bairro["NR_LONGITUDE"].mean()], zoom_start=12)

In [12]:
# 3. Adicione o mapa de calor
heat_data = [[row["NR_LATITUDE"], row["NR_LONGITUDE"], row["QT_ELEITORES_PERFIL"]] for _, row in votos_bairro.iterrows()]
HeatMap(heat_data).add_to(mapa)

In [13]:
mapa.save("mapa_de_calor.html")

In [14]:
mapa